In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00


In [2]:
# Grover's Search Algorithm using Qiskit 2.x

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [3]:
def oracle(qc, n, marked_state):
    """Constructs the oracle for the marked state."""
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)  # use mcx() in Qiskit 2.x
    qc.h(n - 1)
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)

In [4]:
def diffuser(qc, n):
    """Implements the Grover diffuser."""
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    qc.x(range(n))
    qc.h(range(n))

def grover_search(marked_state):
    """Builds Grover's search circuit for a given marked state."""
    n = len(marked_state)
    qc = QuantumCircuit(n, n)
    qc.h(range(n))
    oracle(qc, n, marked_state)
    diffuser(qc, n)
    qc.measure(range(n), range(n))
    return qc

In [5]:
def run_grover(qc):
    """Executes the Grover circuit and displays the result."""
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=1024).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()

if __name__ == "__main__":
    marked_state = "101"
    print("Searching for marked state:", marked_state)
    qc = grover_search(marked_state)
    print(qc.draw(fold=-1))
    run_grover(qc)

Searching for marked state: 101
     ┌───┐          ┌───┐┌───┐               ┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├─────┤M├──────
     ├───┤┌───┐  │  ├───┤├───┤┌───┐       │  ├───┤├───┤     └╥┘┌─┐   
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├───
     ├───┤├───┤┌─┴─┐├───┤├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫──╫─┤M├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║  ║ └╥┘
c: 3/════════════════════════════════════════════════════════╩══╩══╩═
                                                             0  1  2 


### Task 1

In [6]:
# Try different marked states

marked_states_to_try = ["001", "111", "010"]

for marked_state in marked_states_to_try:
    print(f"\nSearching for marked state: {marked_state}")
    qc = grover_search(marked_state)
    # print(qc.draw(fold=-1)) # Uncomment to see the circuit diagram
    run_grover(qc)


Searching for marked state: 001

Searching for marked state: 111

Searching for marked state: 010


### Task 2

In [7]:
def oracle_variable_n(qc, n, marked_state):
    """Constructs the oracle for the marked state for variable n."""
    # Apply X gates to qubits that are 0 in the marked state
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)

    # Apply a multi-controlled Z gate (using H-mcx-H)
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)

    # Apply X gates again to revert the states
    for i, bit in enumerate(marked_state):
        if bit == "0":
            qc.x(i)

In [8]:
def diffuser_variable_n(qc, n):
    """Implements the Grover diffuser for variable n."""
    qc.h(range(n))
    qc.x(range(n))

    # Apply a multi-controlled Z gate (using H-mcx-H)
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)

    qc.x(range(n))
    qc.h(range(n))

In [9]:
def grover_search_variable_n(marked_state):
    """Builds Grover's search circuit for a given marked state and variable n."""
    n = len(marked_state)
    qc = QuantumCircuit(n, n)
    qc.h(range(n))

    # Determine the number of iterations (approx. sqrt(N))
    import numpy as np
    num_iterations = int(np.round(np.pi/4 * np.sqrt(2**n)))

    for _ in range(num_iterations):
        oracle_variable_n(qc, n, marked_state)
        diffuser_variable_n(qc, n)

    qc.measure(range(n), range(n))
    return qc

In [10]:
# Run Grover's search for 4 qubits

marked_state_4_qubits = "1011" # Example marked state for 4 qubits
print("Searching for marked state:", marked_state_4_qubits)
qc_4 = grover_search_variable_n(marked_state_4_qubits)
print(qc_4.draw(fold=-1))
run_grover(qc_4)

Searching for marked state: 1011
     ┌───┐          ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐               ┌───┐┌───┐     ┌─┐         
q_0: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├────────────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├────────────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├─────┤M├─────────
     ├───┤┌───┐  │  ├───┤├───┤┌───┐       │  ├───┤├───┤┌───┐       │  ├───┤├───┤┌───┐       │  ├───┤├───┤┌───┐       │  ├───┤├───┤┌───┐       │  ├───┤├───┤     └╥┘┌─┐      
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├──────
     ├───┤└───┘  │  ├───┤├───┤└───┘       │  ├───┤├───┤└───┘       │  ├───┤├───┤└───┘       │  ├───┤├───┤└───┘       │  ├───┤├───┤└───┘       │  ├───┤├───┤      ║ └╥┘┌─┐   
q_2: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├────────────■──┤ H ├┤ X ├────────────■──┤ X ├┤ 

### Task 3

In [11]:
# Example demonstrating multiple iterations (already included in grover_search_variable_n)

marked_state_multi_iter = "011" # Example marked state
print("Searching for marked state:", marked_state_multi_iter)
qc_multi = grover_search_variable_n(marked_state_multi_iter)
print(qc_multi.draw(fold=-1))
run_grover(qc_multi)

Searching for marked state: 011
     ┌───┐┌───┐     ┌───┐┌───┐┌───┐          ┌───┐┌───┐┌───┐          ┌───┐┌───┐┌───┐          ┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤M├──────
     ├───┤└───┘  │  ├───┤├───┤└───┘       │  ├───┤├───┤└───┘       │  ├───┤├───┤└───┘       │  ├───┤├───┤     └╥┘┌─┐   
q_1: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├────────────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├──────╫─┤M├───
     ├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐┌───┐┌─┴─┐├───┤├───┤┌───┐┌───┐┌─┴─┐├───┤├───┤┌───┐┌───┐┌─┴─┐├───┤├───┤┌───┐ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫──╫─┤M├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║  ║ └╥┘
c: 3/══════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩═
        

### Task 4

In [14]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=76bf47328548ddc670e50a55818540936e8b700f0703e15d25ca60c7a0c6cc86
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [16]:
# Visualize the circuit using qc.draw('text')
# Using the circuit from the last example (marked_state_multi_iter = "011")
print("Visualizing the circuit:")
print(qc_multi.draw('text'))

Visualizing the circuit:
     ┌───┐┌───┐     ┌───┐┌───┐┌───┐          ┌───┐┌───┐┌───┐          ┌───┐»
q_0: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├»
     ├───┤└───┘  │  ├───┤├───┤└───┘       │  ├───┤├───┤└───┘       │  ├───┤»
q_1: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├────────────■──┤ H ├»
     ├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐┌───┐┌─┴─┐├───┤├───┤┌───┐┌───┐┌─┴─┐├───┤»
q_2: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├»
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
c: 3/══════════════════════════════════════════════════════════════════════»
                                                                           »
«     ┌───┐┌───┐          ┌───┐┌───┐     ┌─┐      
«q_0: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤M├──────
«     ├───┤└───┘       │  ├───┤├───┤     └╥┘┌─┐   
«q_1: ┤ X ├────────────■──┤ X ├┤ H ├──────╫─┤M├───
«     ├───┤┌───┐┌───┐┌─┴─┐├───┤├───┤┌───┐ ║ └╥┘┌─┐
«q_2: ┤ H ├┤ X ├┤ H ├┤ X ├┤

### Task 5

In [18]:
# Experiment with Noise

from qiskit_aer.noise import NoiseModel, depolarizing_error

# Create a simple noise model
noise_model = NoiseModel()

# Add depolarizing error to all single qubit gates
single_qubit_error = depolarizing_error(0.01, 1)
noise_model.add_all_qubit_quantum_error(single_qubit_error, ['u1', 'u2', 'u3', 'id', 'rz', 'sx', 'x', 'rx', 'ry']) # Added rx and ry


# Add depolarizing error to all two qubit gates
two_qubit_error = depolarizing_error(0.05, 2)
# Corrected list of 2-qubit gates
noise_model.add_all_qubit_quantum_error(two_qubit_error, ['cx', 'cz', 'cy', 'iswap', 'rxx', 'ryy', 'rzz', 'ecr', 'dcx', 'swap'])


# Get the quantum circuit from the last executed Grover's search (qc_multi)
# If you ran a different circuit last, you might need to adjust this.
qc_noisy = qc_multi.copy() # Create a copy to avoid modifying the original circuit

# Run the circuit with the noisy simulator
noisy_simulator = AerSimulator(noise_model=noise_model)
compiled_circuit_noisy = transpile(qc_noisy, noisy_simulator)
result_noisy = noisy_simulator.run(compiled_circuit_noisy, shots=1024).result()
counts_noisy = result_noisy.get_counts()

print("Results with noise:")
plot_histogram(counts_noisy)
plt.show()

# Compare with ideal results (re-running the ideal simulation for comparison)
print("\nResults from ideal simulator:")
simulator_ideal = AerSimulator()
compiled_circuit_ideal = transpile(qc_multi, simulator_ideal)
result_ideal = simulator_ideal.run(compiled_circuit_ideal, shots=1024).result()
counts_ideal = result_ideal.get_counts()

plot_histogram(counts_ideal)
plt.show()

Results with noise:

Results from ideal simulator:
